In [2]:
from typing import List
from math import pi
import rospy
from copy import deepcopy

from moveit_commander import PlanningSceneInterface

from geometry_msgs.msg import (
    Pose,
    PoseStamped,
    Point,
    Quaternion,
)
from commander.msg import Goal
from commander.srv import (
    ExecuteTrajectory,
    PlanGoal,
    PlanGoalRequest,
    PlanSequence,
    PlanSequenceRequest,
    PickPlace,
    GetTcpPose,
    VisualizePoses,
    SetEe,
)

from commander.utils import poses_from_yaml, load_scene
from commander.transform_utils import orient_poses, create_rotation_matrix, apply_transformation

scene = PlanningSceneInterface()

plan_goal_srv = rospy.ServiceProxy('commander/plan_goal', PlanGoal)
plan_sequence_srv = rospy.ServiceProxy('commander/plan_sequence', PlanSequence)
execute_trajectory_srv = rospy.ServiceProxy('commander/execute_trajectory', ExecuteTrajectory)
get_tcp_pose_srv = rospy.ServiceProxy('commander/get_tcp_pose', GetTcpPose)
set_ee_srv = rospy.ServiceProxy('commander/set_ee', SetEe)
pick_place_srv = rospy.ServiceProxy('commander/pick_place', PickPlace)


def display_poses(poses: List[Pose], frame_id: str = 'base_link') -> None:
    rospy.wait_for_service('/visualize_poses', timeout=10)
    visualize_poses = rospy.ServiceProxy('/visualize_poses', VisualizePoses)
    visualize_poses(frame_id, poses)


rospy.init_node('test')

In [3]:
load_scene()
success = set_ee_srv('rgb_camera_tcp')
sequence = poses_from_yaml('/dev_ws/src/commander/data/a3dp.yaml')
origin = Pose(position=Point(0.775, 0, -0.35), orientation=Quaternion(0, 0, 0, 1))
sequence = orient_poses(origin, sequence)
tf = create_rotation_matrix([0, 0, pi])
sequence = [apply_transformation(pose, tf) for pose in sequence]
home = [0, -pi / 2, pi / 2, 0, pi / 2, pi]
display_poses(sequence)


In [4]:
plan_goal_srv(Goal(joint_values=home, vel_scale=0.1, acc_scale=0.1, planner='ptp'))

success: True
configuration_change: True

In [5]:
execute_trajectory_srv()

success: True

In [12]:
sequence = poses_from_yaml('/dev_ws/src/commander/data/a3dp.yaml')
origin = Pose(position=Point(0.8, 0, -0.30), orientation=Quaternion(0, 0, 0, 1))
sequence = orient_poses(origin, sequence)
tf = create_rotation_matrix([0, 0, pi])
sequence = [apply_transformation(pose, tf) for pose in sequence]
home = [0, -pi / 2, pi / 2, 0, pi / 2, pi]
display_poses(sequence)
start = sequence[0]
sequence = sequence[1:]
resp = plan_goal_srv(Goal(pose=start, vel_scale=0.1, acc_scale=0.1, planner='ptp'))


In [13]:
success = execute_trajectory_srv()


In [14]:
goals = [Goal(pose=pose, vel_scale=0.1, acc_scale=0.01, planner='lin') for pose in sequence]
blends = [0.008] * (len(goals) - 1)
blends.append(0.0)

plan = PlanSequenceRequest(goals=goals, blends=blends)
resp = plan_sequence_srv(plan)

In [15]:
success = execute_trajectory_srv()
